In [4]:
# plotting.py
import h5py
import numpy as np
from synthesizer.conversions import lnu_to_absolute_mag
import pandas as pd
import unyt
from unyt import erg, Hz, s
import cmasher as cmr
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
import sys
import glob

sys.path.append("/home/jovyan/camels/proj1/")
from setup_params import get_photometry, get_luminosity_function, get_colour_distribution, get_safe_name, get_colour_dir_name, get_magnitude_mask
from variables_config import get_config



In [5]:

def get_simulation_color(simulation):
    """Get standard color for each simulation"""
    color_map = {
        "IllustrisTNG": "blue",
        "SIMBA": "green",
        "Astrid": "red",
        "Swift-EAGLE": "orange"
    }
    return color_map.get(simulation, "gray")

def plot_cv_uvlf(input_dir, redshift_label, band, category, simulation):
    """Plot UVLF for CV simulations showing mean, scatter, and individual runs."""
    pattern = f"UVLF_CV_*_{get_safe_name(band)}_{redshift_label}_{category}.txt"
    files = glob.glob(os.path.join(input_dir, pattern))
    
    if not files:
        return None, None, None
    
    all_data = []
    for f in files:
        try:
            df = pd.read_csv(f, sep='\t')
            all_data.append(df)
        except Exception as e:
            continue
    
    if not all_data:
        return None, None, None
    
    magnitudes = all_data[0]['magnitude'].values
    phi_arrays = np.array([df['phi'].values for df in all_data])
    phi_sigma_arrays = np.array([df['phi_sigma'].values for df in all_data])
    
    mean_phi = np.mean(phi_arrays, axis=0)
    std_phi = np.std(phi_arrays, axis=0)
    lower_percentile = np.percentile(phi_arrays, 16, axis=0)
    upper_percentile = np.percentile(phi_arrays, 84, axis=0)
    
    sim_color = get_simulation_color(simulation)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    for phi in phi_arrays:
        ax.plot(magnitudes, phi, '-', color=sim_color, alpha=0.1)
    
    ax.plot(magnitudes, mean_phi, '-', color=sim_color, linewidth=2, 
            label=f'{simulation} Mean UVLF')
    ax.fill_between(magnitudes, lower_percentile, upper_percentile,
                   alpha=0.3, color=sim_color, label='16th-84th Percentile')
    
    ax.errorbar(magnitudes, mean_phi, yerr=std_phi/np.sqrt(len(all_data)),
               fmt='none', ecolor=sim_color, capsize=5)
    
    ax.set_xlabel('M$_{UV}$ [AB mag]', fontsize=12)
    ax.set_ylabel('log$_{10}$ φ [Mpc$^{-3}$ mag$^{-1}$]', fontsize=12)
    ax.set_title(f'{simulation} {band} UVLF\n{category}, {redshift_label}', fontsize=14)
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    return fig, magnitudes, mean_phi

def plot_cv_colors(input_dir, redshift_label, band1, band2, category, simulation):
    """Plot color distributions for CV simulations."""
    filter_system = get_colour_dir_name(band1, band2)
    pattern = f"Colour_CV_*_{filter_system}_{redshift_label}_{category}.txt"
    files = glob.glob(os.path.join(input_dir, pattern))
    
    if not files:
        return None
    
    all_data = []
    for f in files:
        try:
            df = pd.read_csv(f, sep='\t')
            all_data.append(df)
        except Exception as e:
            continue
    
    if not all_data:
        return None
    
    colors = all_data[0]['colour'].values
    dist_arrays = np.array([df['distribution'].values for df in all_data])
    
    mean_dist = np.mean(dist_arrays, axis=0)
    std_dist = np.std(dist_arrays, axis=0)
    lower_percentile = np.percentile(dist_arrays, 16, axis=0)
    upper_percentile = np.percentile(dist_arrays, 84, axis=0)
    
    sim_color = get_simulation_color(simulation)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    n_sims_to_plot = min(5, len(dist_arrays))
    random_indices = np.random.choice(len(dist_arrays), size=n_sims_to_plot, replace=False)
    for idx in random_indices:
        ax.plot(colors, dist_arrays[idx], '-', alpha=0.3, color=sim_color)
    
    ax.plot(colors, mean_dist, '-', color=sim_color, linewidth=2, 
            label=f'{simulation} Mean Distribution')
    ax.fill_between(colors, lower_percentile, upper_percentile,
                   alpha=0.3, color=sim_color, label='16th-84th Percentile')
    
    ax.set_xlabel(f'{band1} - {band2} [mag]', fontsize=12)
    ax.set_ylabel('Normalized Count', fontsize=12)
    ax.set_title(f'{simulation} Color Distribution\n{category}, {redshift_label}', fontsize=14)
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    return fig



In [14]:
def plot_combined_uvlf(all_sims_data, redshifts, output_dir):
    """Create a multi-panel plot showing UVLFs for all simulations at different redshifts."""
    num_redshifts = len(redshifts)
    fig, axes = plt.subplots(1, num_redshifts, figsize=(16, 4))
    if num_redshifts == 1:
        axes = [axes]
    
    for ax, redshift_info in zip(axes, redshifts):
        z = redshift_info['redshift']
        
        for sim_name, sim_data in all_sims_data.items():
            color = get_simulation_color(sim_name)
            
            if z in sim_data:
                z_data = sim_data[z]
                
                # Plot intrinsic (dashed) without label
                if 'intrinsic' in z_data:
                    ax.plot(z_data['intrinsic']['magnitude'], 
                           z_data['intrinsic']['phi'],
                           '--', color=color, alpha=0.5)
                
                # Plot attenuated (solid) with single label for simulation
                if 'attenuated' in z_data:
                    ax.plot(z_data['attenuated']['magnitude'], 
                           z_data['attenuated']['phi'],
                           '-', color=color,
                           label=sim_name)  # Only one label per simulation
        
        ax.set_xlabel('M$_{AB}$', fontsize=10)
        ax.set_ylabel('$\phi$ [Mpc$^{-3}$ dex$^{-1}$]' if ax == axes[0] else '', fontsize=10)
        ax.set_ylim(-5.0, -2.0)
        ax.set_xlim(-26, -17)
        ax.grid(True, alpha=0.3)
        ax.text(0.05, 0.95, f'z = {z}', transform=ax.transAxes, 
                bbox=dict(facecolor='white', alpha=0.8, pad=0.1),
                fontsize=10)
        ax.tick_params(axis='both', which='major', labelsize=8)
    
    # Simpler legend with just simulation names
    legend = axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', 
                          fontsize=8, handlelength=1.5)
    
    plt.subplots_adjust(wspace=0.1)
    plt.savefig(os.path.join(output_dir, 'combined_uvlf.pdf'), 
                bbox_inches='tight', dpi=300)
    plt.close()

In [ ]:
if __name__ == "__main__":
    simulations = ["IllustrisTNG", "SIMBA", "Astrid", "Swift-EAGLE"]
    
    # Dictionary to store data for all simulations
    all_data = {}
    
    for simulation in simulations:
        sim_data = {}
        config = get_config(dataset="CV", simulation=simulation)
        
        # Process each redshift
        for snap, redshift_info in config["redshift_values"].items():
            z_data = {}
            
            # Process each category (intrinsic/attenuated) and band
            for category in ['intrinsic', 'attenuated']:
                for band in config["filters"][category]:
                    filter_system = get_safe_name(band, filter_system_only=True)
                    data_dir = os.path.join(config["lf_data_dir"][category][filter_system],
                                          get_safe_name(redshift_info['label']))
                    
                    # Create individual plot
                    fig, magnitudes, mean_phi = plot_cv_uvlf(
                        input_dir=data_dir,
                        redshift_label=redshift_info['label'],
                        band=band,
                        category=category,
                        simulation=simulation
                    )
                    
                    # Save individual plot if created
                    if fig is not None:
                        plot_output_dir = os.path.join(config["plots_dir"], simulation, 
                                                     "individual", "UVLFs",
                                                     category, get_safe_name(band))
                        os.makedirs(plot_output_dir, exist_ok=True)
                        output_file = os.path.join(
                            plot_output_dir,
                            f"UVLF_{simulation}_{get_safe_name(band)}_{redshift_info['label']}_{category}.pdf"
                        )
                        fig.savefig(output_file, bbox_inches='tight', dpi=300)
                        plt.close(fig)
                    
                    # Store data for combined plot
                    if magnitudes is not None and mean_phi is not None:
                        if category not in z_data:
                            z_data[category] = {}
                        z_data[category]['magnitude'] = magnitudes
                        z_data[category]['phi'] = mean_phi
            
            # Store redshift data
            sim_data[redshift_info['redshift']] = z_data
            
            # Process colors for this redshift
            for category in ['intrinsic', 'attenuated']:
                for band1, band2 in config["colour_pairs"]:
                    filter_system = get_colour_dir_name(band1, band2)
                    color_dir = os.path.join(config["colour_data_dir"][category],
                                           filter_system,
                                           get_safe_name(redshift_info['label']))
                    
                    # Create color plot
                    fig = plot_cv_colors(
                        input_dir=color_dir,
                        redshift_label=redshift_info['label'],
                        band1=band1,
                        band2=band2,
                        category=category,
                        simulation=simulation
                    )
                    
                    # Save color plot if created
                    if fig is not None:
                        plot_output_dir = os.path.join(config["plots_dir"], simulation, 
                                                     "individual", "colors",
                                                     category, filter_system)
                        os.makedirs(plot_output_dir, exist_ok=True)
                        output_file = os.path.join(
                            plot_output_dir,
                            f"Color_{simulation}_{filter_system}_{redshift_info['label']}_{category}.pdf"
                        )
                        fig.savefig(output_file, bbox_inches='tight', dpi=300)
                        plt.close(fig)
        
        # Store simulation data
        all_data[simulation] = sim_data
    
    # Create combined plot
    config = get_config(dataset="CV", simulation=simulations[0])  # Use first sim for config
    redshifts = sorted(config["redshift_values"].values(), key=lambda x: x['redshift'])
    plot_combined_uvlf(all_data, redshifts, config["plots_dir"])

In [ ]:
def plot_combined_colors(all_sims_data, redshifts, output_dir, color_pairs):
    """Create a multi-panel plot showing color distributions for all simulations."""
    num_pairs = len(color_pairs)
    fig, axes = plt.subplots(1, num_pairs, figsize=(16, 4))
    if num_pairs == 1:
        axes = [axes]
    
    for ax, (band1, band2) in zip(axes, color_pairs):
        for sim_name, sim_data in all_sims_data.items():
            color = get_simulation_color(sim_name)
            sim_label = sim_name.replace("-EAGLE", "")
            
            # Plot intrinsic (dashed)
            if 'intrinsic' in sim_data:
                ax.plot(sim_data['intrinsic']['colour'], 
                       sim_data['intrinsic']['distribution'],
                       '--', color=color, alpha=0.5)
            
            # Plot attenuated (solid)
            if 'attenuated' in sim_data:
                ax.plot(sim_data['attenuated']['colour'], 
                       sim_data['attenuated']['distribution'],
                       '-', color=color, label=sim_label)
        
        ax.set_xlabel(f'{band1} - {band2} [mag]', fontsize=10)
        ax.set_ylabel('Normalized Count' if ax == axes[0] else '', fontsize=10)
        ax.set_title(f'{band1} - {band2}', fontsize=10)
        ax.grid(True, alpha=0.3)
        ax.tick_params(axis='both', which='major', labelsize=8)
    
    # Add legend to the first panel
    legend = axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', 
                          fontsize=8, handlelength=1.5)
    
    plt.subplots_adjust(wspace=0.1)
    plt.savefig(os.path.join(output_dir, 'combined_colors.pdf'), 
                bbox_inches='tight', dpi=300)
    plt.close()

def plot_cv_colors(input_dir, redshift_label, band1, band2, category, simulation):
    """Plot individual color distributions for CV simulations."""
    filter_system = get_colour_dir_name(band1, band2)
    pattern = f"Colour_CV_*_{filter_system}_{redshift_label}_{category}.txt"
    files = glob.glob(os.path.join(input_dir, pattern))
    
    if not files:
        return None, None, None
    
    all_data = []
    for f in files:
        try:
            df = pd.read_csv(f, sep='\t')
            all_data.append(df)
        except Exception as e:
            continue
    
    if not all_data:
        return None, None, None
    
    colors = all_data[0]['colour'].values
    dist_arrays = np.array([df['distribution'].values for df in all_data])
    
    mean_dist = np.mean(dist_arrays, axis=0)
    std_dist = np.std(dist_arrays, axis=0)
    
    sim_color = get_simulation_color(simulation)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Plot individual runs with low alpha
    for dist in dist_arrays:
        ax.plot(colors, dist, '-', alpha=0.1, color=sim_color)
    
    # Plot mean with solid line
    ax.plot(colors, mean_dist, '-', color=sim_color, linewidth=2, 
            label=simulation)
    
    ax.set_xlabel(f'{band1} - {band2} [mag]', fontsize=12)
    ax.set_ylabel('Normalized Count', fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    return fig, colors, mean_dist

# Modify the main script to include color plotting
if __name__ == "__main__":
    simulations = ["IllustrisTNG", "SIMBA", "Astrid", "Swift-EAGLE"]
    
    # Dictionary to store all data
    all_data = {}
    
    for simulation in simulations:
        config = get_config(dataset="CV", simulation=simulation)
        
        # Process each color pair
        for band1, band2 in config["colour_pairs"]:
            color_data = {}
            
            for category in ['intrinsic', 'attenuated']:
                filter_system = get_colour_dir_name(band1, band2)
                data_dir = os.path.join(config["colour_data_dir"][category],
                                      filter_system)
                
                fig, colors, mean_dist = plot_cv_colors(
                    input_dir=data_dir,
                    redshift_label=config["redshift_values"]['label'],
                    band1=band1,
                    band2=band2,
                    category=category,
                    simulation=simulation
                )
                
                if fig is not None:
                    # Save individual plot
                    plot_output_dir = os.path.join(config["plots_dir"], simulation,
                                                 "individual", "colors",
                                                 category, filter_system)
                    os.makedirs(plot_output_dir, exist_ok=True)
                    fig.savefig(os.path.join(plot_output_dir, 
                              f"Color_{simulation}_{filter_system}_{category}.pdf"),
                              bbox_inches='tight', dpi=300)
                    plt.close(fig)
                
                if colors is not None and mean_dist is not None:
                    color_data[category] = {
                        'colour': colors,
                        'distribution': mean_dist
                    }
            
            if color_data:
                all_data[simulation] = color_data
    
    # Create combined color plots
    plot_combined_colors(all_data, 
                        config["redshift_values"],
                        config["plots_dir"],
                        config["colour_pairs"])